## Bibliotecas

In [2]:
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display
import toml
import gspread
from google.oauth2.service_account import Credentials
from auxiliar.arrumar_bases import *

def get_sheetdata(sheet_name):
    # Load credentials from the .streamlit/secrets.toml file
    secrets = toml.load(".streamlit/secrets.toml")
    creds_info = secrets["connections"]["gsheets"]

    # Define scopes for accessing Google Sheets and Drive
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    # Create credentials and authorize the gspread client
    creds = Credentials.from_service_account_info(creds_info, scopes=scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet using the URL from the toml file
    spreadsheet = client.open_by_url(creds_info["spreadsheet"])

    # Select the first worksheet and fetch all records
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()

    df = pd.DataFrame(data)

    return df
    
vmb_path = "assets/venda-mensal-bruta_colab-bi_2025-02-11.xlsx"
agendamento_path = "assets/agendamentos_colab-bi_2025-02-12 (1).xlsx"

venda_mensal_df = pd.read_excel(vmb_path)
agendamentos_df = pd.read_excel(agendamento_path)
procedimentos_df = get_sheetdata("procedimentos_padronizados")
prestadora_df = get_sheetdata("base_prestadoras")
comissao_df = get_sheetdata("comissoes")
tipo_prestadora_df = get_sheetdata("tipo_prestadora")



##Visualizar

In [3]:

base_compilada = criar_base_compilada(agendamentos_df,venda_mensal_df,procedimentos_df,prestadora_df,comissao_df,tipo_prestadora_df)
periodos = base_compilada["periodo"].unique()
periodos = pd.Series(periodos)
s_date_str = "periodo: " + periodos.dt.strftime('%Y-%m-%d')


base_compilada.columns

base_comissoes = criar_comissoes(base_compilada)
display(base_comissoes)


base_avaliacoes = cria_avaliacoes(base_compilada,"mensal")
display(base_avaliacoes)

base_final = juntar_bases(base_comissoes,base_avaliacoes)
display(base_final)

,nome_prestadora,tipo_prestadora,Unidade,comissao_total
0,agda mikie ribeiro mikami,salão parceiro,campinas,9654.0
1,alice saccaro,prestação de serviços - biomedica,ipiranga,1101.0
2,alice saccaro,prestação de serviços - biomedica,mooca,498.0
3,aline barbosa oliveira saragoça,prestação de serviços - biomedica,santo amaro,432.0
4,amanda cristina soares da silva,salão parceiro,tijuca,9224.0
...,...,...,...,...
90,tassia ferreira ribeiro de oliveira,prestação de serviços - biomedica,santos,640.0
91,tatiana cristina bartolomeu angelo ricardo,prestação de serviços - biomedica,campinas,1489.0
92,tatiana cristina bartolomeu angelo ricardo,prestação de serviços - biomedica,jardins,4582.0
93,thais nayara da silveira pires,salão parceiro,londrina,11499.0


avaliacoes_total
nome_prestadora                            tipo_prestadora                   Unidade                      
agda mikie ribeiro mikami                  salão parceiro                    campinas                  270
alice saccaro                              prestação de serviços - biomedica ipiranga                  120
                                                                             mooca                     160
aline barbosa oliveira saragoça            prestação de serviços - biomedica santo amaro                20
amanda dos santos                          salão parceiro                    mooca                     140
...                                                                                                    ...
samantha batista borges nogueira           salão parceiro                    copacabana                200
stefani michailiszen                       prestação de serviços - biomedica jardins                    10
tatiana cristina bartolomeu angelo ricardo prestação de serviços - biomedica campinas                   90
                                                                             jardins                    40
thais nayara da silveira pires             salão parceiro                    londrina                   90

[69 rows x 1 columns]

,nome_prestadora,tipo_prestadora,Unidade,comissao_total,avaliacoes_total
0,agda mikie ribeiro mikami,salão parceiro,campinas,9654.0,270.0
1,alice saccaro,prestação de serviços - biomedica,ipiranga,1101.0,120.0
2,alice saccaro,prestação de serviços - biomedica,mooca,498.0,160.0
3,aline barbosa oliveira saragoça,prestação de serviços - biomedica,santo amaro,432.0,20.0
4,amanda cristina soares da silva,salão parceiro,tijuca,9224.0,NaN
...,...,...,...,...,...
90,tassia ferreira ribeiro de oliveira,prestação de serviços - biomedica,santos,640.0,NaN
91,tatiana cristina bartolomeu angelo ricardo,prestação de serviços - biomedica,campinas,1489.0,90.0
92,tatiana cristina bartolomeu angelo ricardo,prestação de serviços - biomedica,jardins,4582.0,40.0
93,thais nayara da silveira pires,salão parceiro,londrina,11499.0,90.0
